# Buckling under radial force: realistic (beam) spokes

In [ ]:
import bikewheellib as bl
from doetools import RadialBucklingDOE, wheel_library
from shutil import copy2
from imp import find_module

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib inline

In [ ]:
# Set default colors
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                                                    '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                                                    '#bcbd22', '#17becf'])

## ABAQUS numerical experiment

In [ ]:
beam_dir = '../data/doe/doe_rad_buckling_beam'

if True:
    doe = RadialBucklingDOE(out_dir=beam_dir,
                            opts={'spk_paired': False,
                                  'spk_eltype': 'beam',
                                  'sim_type': 'exp',
                                  'sim_u2': 0.01})

    wheel_types = ['cheap_MTB', 'vintage_rd', 'racing_700',
                   'folding', 'track', 'tandem', 'high_wheel']

    for w_type in wheel_types:
        
        wheel = wheel_library.create_predefined_wheel(w_type)
        
        Tc, nc = bl.calc_buckling_tension(wheel)

        for Tn in [0.0, 0.10, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]:
            
            wheel.apply_tension(Tn*Tc)
            
            jobname = '{:s}_{:.2f}'.format(w_type, Tn)
            doe.add_experiment(wheel, opts={'jobname': jobname,
                                            'wheel_type': w_type,
                                            'spk_Tn': Tn,
                                            'spk_T': Tn * Tc})

    print('\nCreated {0:d} simulations'.format(len(doe.db)))

    doe.write_input_files(N_batches=4)
    doe.to_csv()
    
    # Copy postprocessing script
    copy2(src=find_module('doetools')[1] + '/postproc_rad_buckling.py',
          dst=beam_dir)